In [134]:
import pandas as pd
import numpy as np
import holidays
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime

In [3]:
df_economy = pd.read_csv("../data/economy.csv")
df_business = pd.read_csv("../data/business.csv")

In [4]:
print(df_economy.shape)
print(df_business.shape)

(206774, 11)
(93487, 11)


In [5]:
print(df_economy.info())
print(df_business.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206774 entries, 0 to 206773
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   date        206774 non-null  object
 1   airline     206774 non-null  object
 2   ch_code     206774 non-null  object
 3   num_code    206774 non-null  int64 
 4   dep_time    206774 non-null  object
 5   from        206774 non-null  object
 6   time_taken  206774 non-null  object
 7   stop        206774 non-null  object
 8   arr_time    206774 non-null  object
 9   to          206774 non-null  object
 10  price       206774 non-null  object
dtypes: int64(1), object(10)
memory usage: 17.4+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93487 entries, 0 to 93486
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        93487 non-null  object
 1   airline     93487 non-null  object
 2   ch_code     93487 non-nul

In [6]:
df_economy.head(2)

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price
0,11-02-2022,SpiceJet,SG,8709,18:55,Delhi,02h 10m,non-stop,21:05,Mumbai,"5,953"
1,11-02-2022,SpiceJet,SG,8157,06:20,Delhi,02h 20m,non-stop,08:40,Mumbai,"5,953"
2,11-02-2022,AirAsia,I5,764,04:25,Delhi,02h 10m,non-stop,06:35,Mumbai,"5,956"
3,11-02-2022,Vistara,UK,995,10:20,Delhi,02h 15m,non-stop,12:35,Mumbai,"5,955"
4,11-02-2022,Vistara,UK,963,08:50,Delhi,02h 20m,non-stop,11:10,Mumbai,"5,955"


In [7]:
df_business.head(2)

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price
0,11-02-2022,Air India,AI,868,18:00,Delhi,02h 00m,non-stop,20:00,Mumbai,"25,612"
1,11-02-2022,Air India,AI,624,19:00,Delhi,02h 15m,non-stop,21:15,Mumbai,"25,612"
2,11-02-2022,Air India,AI,531,20:00,Delhi,24h 45m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:45,Mumbai,"42,220"
3,11-02-2022,Air India,AI,839,21:25,Delhi,26h 30m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,23:55,Mumbai,"44,450"
4,11-02-2022,Air India,AI,544,17:15,Delhi,06h 40m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,23:55,Mumbai,"46,690"


## Pre processing Data

This project aims to predict the price of a flight ticket based on the information provided by the user. The data was obtained from Kaggle and it is composed of two datasets, one with the data from business flights and the other with the data from economy flights. The data is available in the following link: [Kaggle](https://www.kaggle.com/datasets/shubhambathwal/flight-price-prediction).

They already pre processed a dataset and make it available for download, but I decided to do it myself to get more familiar with the data and to practice my data cleaning skills.

Some of thesteps that I'll do to clean the data are:

- Cast the data from both datasets, classify the flights (economy or business) and join them;
- Remove duplicates;
- Create a flight code based on the characters and numbers
- Classify stops;
- Classify the time of the day;
- Classify the day of the week;
- Classify if the flight is a holiday;
- Create a new column with the days until the flight;
- Duration of the flight to continuous.


### Cast the data from both datasets, classify the flights (economy or business) and join them


In [84]:
df_economy["price"] = df_economy["price"].str.replace(",", "").astype(int)
df_business["price"] = df_business["price"].str.replace(",", "").astype(int)

AttributeError: Can only use .str accessor with string values!

In [143]:
df_economy["class"] = "economy"
df_business["class"] = "business"
df = pd.concat([df_economy, df_business], ignore_index=True)

### Remove duplicates


In [144]:
df.shape

(300261, 12)

In [145]:
df = df.drop_duplicates()

In [146]:
df.shape

(300259, 12)

### Create a flight code based on the characters and numbers


In [147]:
df["flight_code"] = df["ch_code"] + df["num_code"].astype(str)

In [148]:
df = df.drop(columns=["ch_code", "num_code"])

### Classify stops


In [149]:
df["stop"].str.split("-")

0                                              [non, stop ]
1                                              [non, stop ]
2                                              [non, stop ]
3                                              [non, stop ]
4                                              [non, stop ]
                                ...                        
300256    [1, stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t...
300257    [1, stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t...
300258    [1, stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t...
300259    [1, stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t...
300260    [1, stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t...
Name: stop, Length: 300259, dtype: object

In [150]:
df["stops"] = df["stop"].str.split("-").str[0]

In [151]:
df["stops"] = df["stops"].replace("non", "0")

In [152]:
df = df.drop(columns=["stop"])

### Classify the time of the day


In [153]:
early_morning = ["00", "01", "02", "03", "04", "05"]
morning = ["06", "07", "08", "09", "10", "11"]
afternoon = ["12", "13", "14", "15", "16", "17"]
evening = ["18", "19", "20", "21", "22", "23"]

In [154]:
df['departure_time'] = df['dep_time'].str.split(':').str[0]
df['arrival_time'] = df['arr_time'].str.split(':').str[0]

df['departure_time'] = df['departure_time'].replace(
    early_morning, 'early_morning')
df['departure_time'] = df['departure_time'].replace(morning, 'morning')
df['departure_time'] = df['departure_time'].replace(afternoon, 'afternoon')
df['departure_time'] = df['departure_time'].replace(evening, 'evening')

df['arrival_time'] = df['arrival_time'].replace(early_morning, 'early_morning')
df['arrival_time'] = df['arrival_time'].replace(morning, 'morning')
df['arrival_time'] = df['arrival_time'].replace(afternoon, 'afternoon')
df['arrival_time'] = df['arrival_time'].replace(evening, 'evening')

In [155]:
df = df.drop(columns=["dep_time", "arr_time"])

### Classify the day of the week


In [156]:
df["datetime"] = pd.to_datetime(df["date"], dayfirst=True)

In [157]:
df["dow"] = df["datetime"].dt.day_name()

### Classify if the flight is a holiday


In [158]:
indian_holidays = holidays.India()

In [159]:
indian_holidays.values()

dict_values([])

In [160]:
df["holiday"] = df["date"].apply(lambda x: indian_holidays.get(x))

In [161]:
pd.isnull(df["holiday"]).value_counts()

holiday
True     287363
False     12896
Name: count, dtype: int64

In [162]:
df['holiday'] = df['holiday'].fillna(0)
df.loc[df['holiday'] != 0, 'holiday'] = 1

In [163]:
df["holiday"].value_counts()

holiday
0    287363
1     12896
Name: count, dtype: int64

### Create a new column with the days until the flight


In [164]:
df.sort_values(by=["datetime"]).head(2)

,date,airline,from,time_taken,to,price,class,flight_code,stops,departure_time,arrival_time,datetime,dow,holiday
0,11-02-2022,SpiceJet,Delhi,02h 10m,Mumbai,5953,economy,SG8709,0,evening,evening,2022-02-11,Friday,0
127595,11-02-2022,Vistara,Kolkata,25h 10m,Mumbai,16044,economy,UK778,1,afternoon,afternoon,2022-02-11,Friday,0


In [165]:
df['days_until'] = (df['datetime'] - datetime(2022, 2, 10)).dt.days

In [166]:
df.head(3)

,date,airline,from,time_taken,to,price,class,flight_code,stops,departure_time,arrival_time,datetime,dow,holiday,days_until
0,11-02-2022,SpiceJet,Delhi,02h 10m,Mumbai,5953,economy,SG8709,0,evening,evening,2022-02-11,Friday,0,1
1,11-02-2022,SpiceJet,Delhi,02h 20m,Mumbai,5953,economy,SG8157,0,morning,morning,2022-02-11,Friday,0,1
2,11-02-2022,AirAsia,Delhi,02h 10m,Mumbai,5956,economy,I5764,0,early_morning,morning,2022-02-11,Friday,0,1


### Duration of the flight to continuous


In [178]:
df["duration"] = df["time_taken"].str.replace("h", ":").str.replace("m", "")
df["duration"] = df["duration"].str.split(":")
df["duration_hours"] = df["duration"].str[0]
df["duration_minutes"] = df["duration"].str[1]

In [179]:
df["duration_hours"].unique()

array(['02', '12', '16', '11', '14', '15', '03', '05', '08', '06', '18',
       '23', '24', '04', '19', '22', '26', '17', '20', '21', '07', '10',
       '28', '09', '13', '29', '27', '30', '25', '31', '33', '36', '35',
       '34', '39', '01', '37', '40', '32', '7', '41', '38', '1.03',
       '1.02', '2', '00', '47', '1.01', '5', '42', '49', '45', '44'],
      dtype=object)

In [175]:
df.loc[df['duration_hours'] == '1.02', :]

,date,airline,from,time_taken,to,price,class,flight_code,stops,departure_time,arrival_time,datetime,dow,holiday,days_until,duration,duration_hours,duration_minutes
104676,26-02-2022,GO FIRST,Bangalore,1.02h m,Kolkata,5177,economy,G8146,2+,early_morning,morning,2022-02-26,Saturday,0,16,"[1.02, ]",1.02,


In [176]:
df.loc[df['duration_hours'] == '1.01', :]

,date,airline,from,time_taken,to,price,class,flight_code,stops,departure_time,arrival_time,datetime,dow,holiday,days_until,duration,duration_hours,duration_minutes
154007,25-02-2022,GO FIRST,Hyderabad,1.01h m,Delhi,6132,economy,G8517,1,evening,evening,2022-02-25,Friday,0,15,"[1.01, ]",1.01,


In [182]:
df["duration_hours"] = [None if "." in x else x for x in df["duration_hours"]]

In [184]:
df['duration'] = pd.to_numeric(df['duration_hours'], errors='coerce') * \
    60 + pd.to_numeric(df['duration_minutes'], errors='coerce')

In [188]:
df = df.drop(columns=["duration_hours", "duration_minutes", "time_taken"])

KeyError: "['duration_hours', 'duration_minutes'] not found in axis"